In [1]:
%run ../0_Config.ipynb

In [2]:
%run ../0_UDFs.ipynb

In [3]:
import numpy as np
from pathlib import Path

In [4]:
app_config

{'raw_data': {'date_folder': '03082023',
  'files': {'current_time': 'current_time.dat',
   'events': 'events.xlsx',
   'orig_comp_and_level_desc': 'orig_comp_and_level_desc.dat',
   'planner_codes': 'planner_codes.xlsx',
   'price_xdock_promo_attribs': 'price_xdock_promo_attribs.csv',
   'sas_time': 'sas_time.xlsx',
   'tds_override': 'tds_override.dat',
   'tot_sales': 'tot_sales.dat'}},
 'dependent_variable': 'Sales_Qty_fill',
 'date_var': 'Week_Start_Date',
 'date_format_pandas': '%d-%m-%Y',
 'modeling_granularity': ['Div_No', 'Store_No', 'Base_UPC'],
 'input_file_path': '/home/satish/Modelling/Forecaster/modeling-sample.csv',
 'output_dir_path': '/home/satish/Modelilng/Forecaster/modeling_results_python',
 'data_processing': {'outlier_treatment_needed': True,
  'outlier_treatment': {'algorithm': 'inter_quartile_range',
   'hampel_filter': {'window_size': 2, 'no_of_sigmas': 3},
   'inter_quartile_range': {'minimum_quantile': 5, 'maximum_quantile': 95}},
  'feature_engineering': {'p

In [5]:
data_path = "../../../../data/raw/"
raw_data_related_config = app_config['raw_data']
folder_datestamp = raw_data_related_config['date_folder']

print(folder_datestamp)
print(os.listdir(data_path + folder_datestamp + '/'))

output_parts = data_path + folder_datestamp + '/by_bindc/'
Path(output_parts).mkdir(exist_ok=True, parents=True)

03082023
['by_bindc', 'current_time.dat', 'events.xlsx', 'orig_comp_and_level_desc.dat', 'planner_codes.xlsx', 'price_xdock_promo_attribs.csv', 'price_xdock_promo_attribs.xlsx', 'sas_time.xlsx', 'tds_override.dat', 'tot_sales.dat']


# UDFs

In [6]:
def filter_promo(data, measure_names_dict, filter_column, filter_column_values):

    filter_df = data
    
    for measure_name in measure_names_dict.keys():
        if filter_column in measure_names_dict[measure_name]:
            filter_df_non_measure = filter_df[filter_df['Measure_name']!=measure_name].copy()
            filter_df_measure = filter_df[filter_df['Measure_name']==measure_name].copy()
            filter_df_measure = filter_df_measure[filter_df_measure[filter_column].isin(filter_column_values)].copy()

            filter_df = pd.concat([filter_df_non_measure, filter_df_measure], axis = 0 , ignore_index=True)
    
    return filter_df

In [7]:
def get_prepared_data(data, measure_names_dict):
    data['tot_sales']['Measure_name'] = 'tot_sales'
    data['tds_override']['Measure_name'] = 'tds_override'

    appended_data = pd.concat([data['price_xdock_promo_attribs'], data['tot_sales'],data['tds_override']]
    , axis = 0
    , ignore_index = True, sort = False, join = 'outer')

    measure_names = appended_data['Measure_name'].drop_duplicates().to_list()

    for measure_name in measure_names[measure_names.index('tot_sales'):] + measure_names[:measure_names.index('tot_sales')]:
        wk_cols = [col for col in appended_data.columns if '_Wk_' in col]
        measure_col = 'Measure_name' 
        try:
            id_vars_ = measure_names_dict[measure_name]
        except:
            id_vars_ = [col for col in appended_data.columns if col not in wk_cols + [measure_col]]

        out_ = pd.melt(appended_data[appended_data['Measure_name']==measure_name].copy()
                    ,id_vars = id_vars_
                    ,value_vars = wk_cols
                    ,var_name = 'Pd_Wk_Yr'
                    ,value_name= measure_name)
        
        if measure_name != 'tot_sales':
            out_final = out_final.merge(out_
                                        ,on = id_vars_ + ['Pd_Wk_Yr']
                                        ,how = 'left')
            assert out_final.shape[0] == out_final_before[0], "Duplicates exist"
        else:
            out_final = out_
            out_final_before = out_final.shape


    out_final_w_events = out_final.merge(data['events'], on = ['Pd_Wk_Yr'], how = 'left')

    assert out_final_w_events.shape[0] == out_final_before[0], "Duplicates exist"

    return out_final_w_events

# Reading input files

## Reading Dat Files

In [8]:
current_week = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['current_time']).columns[0]
orig_comp_and_level_desc = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['orig_comp_and_level_desc']
                                        , {'sep' : ':|\\|', 'encoding' : 'unicode_escape','engine':'python'})

tds_override = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['tds_override']
                                        , {'sep' : '|'})

try:
    tot_sales = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['tot_sales']
                            , {'sep' : '|'})
except:
    n_columns = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['tot_sales']
                            , {'sep' : '|', 'nrows' : 2}).columns
    tot_sales = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['tot_sales']
                            , {'sep' : '|', 'usecols' : range(len(n_columns))})    
    


In [9]:
tot_sales = tot_sales[tot_sales['Account']==23].copy()
tds_override = tds_override[tds_override['Account'] == 23].copy()

## Reading Excel files

In [10]:
events = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['events'])
planner_codes = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['planner_codes'])
sas_time = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['sas_time'])
# price_xdock_promo_attribs = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['price_xdock_promo_attribs'])
# price_xdock_promo_attribs.to_csv(data_path + folder_datestamp +'/' + raw_data_related_config['files']['price_xdock_promo_attribs'].replace('.xlsx','.csv'), index=False)


## Reading csv files

In [11]:
price_xdock_promo_attribs = data_loading(data_path + folder_datestamp +'/' + raw_data_related_config['files']['price_xdock_promo_attribs'])
# price_xdock_promo_attribs = price_xdock_promo_attribs[price_xdock_promo_attribs['Account'] == 23].copy()

# Selecting BinDCs x Whse x BDCs due to data constraint

In [12]:
week_2022_cols = [col_ for col_ in tot_sales.columns if '_Wk_' in col_ and  col_.endswith('_22')]
tot_sales['sales_2022'] = tot_sales[week_2022_cols].sum(axis = 1)
tot_sales_select = tot_sales[tot_sales['sales_2022'] > 0].drop(columns = [col_ for col_ in tot_sales.columns if '_Wk_' in col_]).copy()
tot_sales_select.reset_index(drop=True,inplace=True)

In [13]:
cum_sales_by_whse = tot_sales\
    .groupby(['Whse'])\
        .agg({'sales_2022' : ['sum','count']})\
        .reset_index(drop = False)

cum_sales_by_whse.columns = ['_'.join(i) if i[1] != '' else i[0] for i in cum_sales_by_whse.columns]
cum_sales_by_whse.sort_values(['sales_2022_count'], ascending=[False], inplace = True)

cum_sales_by_whse['cumsum_sales_2022'] = cum_sales_by_whse['sales_2022_sum'].cumsum() / cum_sales_by_whse['sales_2022_sum'].sum()
cum_sales_by_whse

,Whse,sales_2022_sum,sales_2022_count,cumsum_sales_2022
31,32320,174497846,33484,0.175061
16,4330,106031429,22624,0.281434
7,2370,111797440,20745,0.393593
30,17483,61950719,12716,0.455743
22,4567,27446456,11964,0.483278
27,12270,45636914,9542,0.529062
2,2161,54248009,9203,0.583485
14,2557,48336492,5947,0.631978
1,2114,32593565,4491,0.664677
9,2421,50609247,4463,0.715449


In [14]:
cum_sales_by_bindc = tot_sales\
    .groupby(['BinDC'])\
        .agg({'sales_2022' : ['sum','count']})\
        .reset_index(drop = False)

cum_sales_by_bindc.columns = ['_'.join(i) if i[1] != '' else i[0] for i in cum_sales_by_bindc.columns]
cum_sales_by_bindc.sort_values(['sales_2022_count'], ascending=[False], inplace = True)

cum_sales_by_bindc['cumsum_sales_2022'] = cum_sales_by_bindc['sales_2022_sum'].cumsum() / cum_sales_by_bindc['sales_2022_sum'].sum()
cum_sales_by_bindc

,BinDC,sales_2022_sum,sales_2022_count,cumsum_sales_2022
102,17483,61950719,12716,0.062151
92,12270,45636914,9542,0.107935
12,2161,54248009,9203,0.162358
103,44330,61027813,7017,0.223583
55,2557,48336492,5947,0.272075
...,...,...,...,...
44,2531,0,362,0.999183
82,4646,169599,331,0.999353
115,301150,340222,324,0.999695
25,2463,114734,321,0.999810


In [15]:
cum_sales_by_bdc = tot_sales\
    .groupby(['BDC'])\
        .agg({'sales_2022' : ['sum','count']})\
        .reset_index(drop = False)

cum_sales_by_bdc.columns = ['_'.join(i) if i[1] != '' else i[0] for i in cum_sales_by_bdc.columns]
cum_sales_by_bdc.sort_values(['sales_2022_count'], ascending=[False], inplace = True)

cum_sales_by_bdc['cumsum_sales_2022'] = cum_sales_by_bdc['sales_2022_sum'].cumsum() / cum_sales_by_bdc['sales_2022_sum'].sum()
cum_sales_by_bdc

,BDC,sales_2022_sum,sales_2022_count,cumsum_sales_2022
215,5030053,4403,426,0.000004
523,20026943,0,426,0.000004
449,15026033,3957485,425,0.003975
172,3026009,3059820,425,0.007044
536,23026019,4439865,425,0.011499
...,...,...,...,...
751,90047523,0,1,1.000000
284,10047500,0,1,1.000000
225,6450030,-5,1,1.000000
680,85004865,0,1,1.000000


In [16]:
tot_sales.drop(columns = ['sales_2022'], inplace=True)

In [17]:
id_vars = ['Event_Code','Description','Week_Before','Week_Of','Week_After','Usage']
week_cols = [col for col in events.columns if '_Wk_' in col]
var_name = 'Pd_Wk_Yr'

In [18]:
events_final = pd.melt(events
                             , id_vars=id_vars
                             , value_vars=week_cols
                             , var_name = var_name
                             , value_name = 'event_ind'
                             , ignore_index = True)
events_final[['Week_Before','Week_Of','Week_After']] = events_final[['Week_Before','Week_Of','Week_After']]\
    .apply(lambda x: np.where(x == "x",1,0))
events_final['Pd_Wk_Yr_ver1'] = events_final['Pd_Wk_Yr'].apply(lambda x: x.split('_')).apply(lambda x: '_'.join(x[3:] + x[:3]))

events_final['event_ind_Week_Before'] = (events_final\
    .sort_values(['Pd_Wk_Yr_ver1'], ascending = False)\
    .groupby(id_vars)['event_ind']\
        .shift(1) * events_final['Week_Before']).fillna(0)
events_final['event_ind_Week_After'] = (events_final\
    .sort_values(['Pd_Wk_Yr_ver1'], ascending = False)\
    .groupby(id_vars)['event_ind']\
        .shift(-1) * events_final['Week_After']).fillna(0)

events_final['event_ind_Week_Of'] = events_final['event_ind'] * events_final['Week_Of']
events_final.drop(columns = ['Pd_Wk_Yr_ver1'], inplace=True)

# events_final[events_final['Description']=='Super Bowl'].head(7)

events_final = events_final.pivot_table(index=['Pd_Wk_Yr']
                                        , columns=['Event_Code','Description']
                                        , values=['event_ind_Week_Before','event_ind_Week_After','event_ind_Week_Of'])

events_final.columns = ['_'.join([str(j) for j in i]) for i in events_final.columns]
events_final.reset_index(drop=False, inplace=True)

In [19]:
price_xdock_promo_attribs = pd.concat([price_xdock_promo_attribs, planner_codes]
                                      , axis =0
                                      , ignore_index=True
                                      , join = 'outer'
                                      , sort = False)

In [20]:
measure_names = price_xdock_promo_attribs['Measure_name'].drop_duplicates().to_list()

measure_names_dict = {}

non_wk_cols = [col for col in price_xdock_promo_attribs.columns if '_Wk_' not in col]

for measure in measure_names:
    describe_stats = price_xdock_promo_attribs[price_xdock_promo_attribs['Measure_name']==measure][non_wk_cols].describe().loc['count',]
    measure_names_dict[measure] = describe_stats[describe_stats>0].index.to_list()

In [21]:
measure_names_dict

{'DLFT': ['Account', 'BDC'],
 'APP': ['Account', 'BDC'],
 'OPP': ['Account', 'BDC'],
 'OPD': ['Account', 'BDC'],
 'SRP': ['Whse', 'BinDC', 'Account', 'BDC'],
 'OPDW': ['Account', 'BDC'],
 'APDW': ['Account', 'BDC'],
 'XDOCK': ['Area1',
  'Zone',
  'District',
  'Whse',
  'BinDC',
  'Account',
  'Area2',
  'Category',
  'BU',
  'Class',
  'Size',
  'BDC'],
 'SDV': ['Whse', 'BinDC', 'BDC'],
 'APD': ['Account', 'BDC'],
 'NEW_OR_REF_OR_DISCON': ['BDC']}

In [22]:
price_xdock_promo_attribs_filter_account = filter_promo(price_xdock_promo_attribs.copy()
                                                        , measure_names_dict
                                                        ,'Account'
                                                        ,[23])

In [23]:
price_xdock_promo_attribs_filter_bindc = filter_promo(price_xdock_promo_attribs_filter_account.copy()
                                                      , measure_names_dict
                                                      ,'BinDC'
                                                      ,[17483])

In [24]:
print(price_xdock_promo_attribs.groupby(['Measure_name'])['BDC'].count())
print(price_xdock_promo_attribs_filter_account.groupby(['Measure_name'])['BDC'].count())
print(price_xdock_promo_attribs_filter_bindc.groupby(['Measure_name'])['BDC'].count())

Measure_name
APD                       9371
APDW                      9371
APP                       9371
DLFT                       891
NEW_OR_REF_OR_DISCON       378
OPD                         18
OPDW                        18
OPP                        208
SDV                     185218
SRP                     259470
XDOCK                    45110
Name: BDC, dtype: int64
Measure_name
APD                        701
APDW                       701
APP                        701
NEW_OR_REF_OR_DISCON       378
OPP                        117
SDV                     185218
SRP                      59042
XDOCK                    39043
Name: BDC, dtype: int64
Measure_name
APD                      701
APDW                     701
APP                      701
NEW_OR_REF_OR_DISCON     378
OPP                      117
SDV                      845
SRP                      582
XDOCK                   3040
Name: BDC, dtype: int64


In [25]:
print(price_xdock_promo_attribs.shape)
print(price_xdock_promo_attribs_filter_account.shape)

(519424, 222)
(285901, 222)


In [26]:
# data = {'tot_sales' : tot_sales[tot_sales['BinDC'] == 17483].copy()
#         ,'tds_override' : tds_override[tds_override['BinDC'] == 17483].copy()
#         ,'events' : events_final.copy()
#         ,'price_xdock_promo_attribs' : filter_promo(price_xdock_promo_attribs_filter_account.copy()
#                                                         , measure_names_dict
#                                                         ,'BinDC'
#                                                         ,[17483])
#         ,'planner_codes' :                                                         
#                                                         }
# measure_names_dict

In [27]:
for bindc in tot_sales['BinDC'].unique():
    print(bindc)

    data_dict = {'tot_sales' : tot_sales[tot_sales['BinDC'] == bindc].copy()
                 ,'tds_override' : tds_override[tds_override['BinDC'] == bindc].copy()
                 ,'events' : events_final.copy()
                 ,'price_xdock_promo_attribs' : filter_promo(price_xdock_promo_attribs_filter_account.copy()
                                                             , measure_names_dict
                                                             ,'BinDC'
                                                             ,[bindc])}
    
    prepared_data_dc = get_prepared_data(data_dict, measure_names_dict)

    prepared_data_dc.to_csv(output_parts + str(bindc) + '.csv', index = False)

12270
2559
300106
300342
300123
2171
9937
2124
2512
2525
2543
2057
2511
2115
2113
2764
17433
2145
2544
2502
2516
2529
2518
2535
2519
300398
12278
12223
300181
300122
2017
9969
2241
300411
2161
304598
2114
2557
2148
2042
304873
2121
2102
78844
2404
2435
304648
2421
2403
2417
9172
2464
2445
2401
2471
2436
2443
2463
2449
304073
17346
2108
2619
8506
3756
4630
78765
4628
16421
44330
4643
5729
301150
4646
17446
4645
4648
300086
4624
4625
4413
4420
4414
4401
17483
17475
16580
4404
4124
2506
2528
2550
2765
2551
4629
4644
12253
2521
2552
10186
2513
2533
2510
2532
2531
2527
2537
2509
2507
2540
4407
4432
16518
17322
4405
4412
4417
4419
4435
301031
